In [1]:
import asyncio
import logging
import sys
from concurrent.futures import ThreadPoolExecutor, as_completed
from functools import partial
import time
import os
from datetime import datetime, timedelta
import json
import pickle

import random
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt

current_path = os.getcwd()

sys.path.append(os.path.join(current_path, "../.."))
from helpers.llm_prompts import (
    ZERO_SHOT_MARKET_PROMPT,
    ZERO_SHOT_NON_MARKET_PROMPT,
    ZERO_SHOT_JOINT_QUESTION_PROMPT,
    SCRATCH_PAD_MARKET_PROMPT,
    SCRATCH_PAD_NON_MARKET_PROMPT,
    SCRATCH_PAD_JOINT_QUESTION_PROMPT,
    SCRATCH_PAD_WITH_SUMMARIES_MARKET_PROMPT,
    SCRATCH_PAD_WITH_SUMMARIES_NON_MARKET_PROMPT,
    SCRATCH_PAD_WITH_SUMMARIES_JOINT_QUESTION_PROMPT,
    REFORMAT_SINGLE_PROMPT,
    REFORMAT_PROMPT,
    HUMAN_JOINT_PROMPT_1,
    HUMAN_JOINT_PROMPT_2,
    HUMAN_JOINT_PROMPT_3,
    HUMAN_JOINT_PROMPT_4,
)

from helpers import model_eval


logger = logging.getLogger()
logger.setLevel(logging.INFO)

/Users/apple/envs/myenv-llm-bench/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
human_joint_prompts = [
    HUMAN_JOINT_PROMPT_1,
    HUMAN_JOINT_PROMPT_2,
    HUMAN_JOINT_PROMPT_3,
    HUMAN_JOINT_PROMPT_4,
]

models = {
    "gpt_3p5_turbo_0125": {"source": "OAI", "full_name": "gpt-3.5-turbo-0125"},
    "gpt_4": {"source": "OAI", "full_name": "gpt-4"},
    "gpt_4_turbo_0409": {"source": "OAI", "full_name": "gpt-4-turbo-2024-04-09"},
    "gpt_4_1106_preview": {"source": "OAI", "full_name": "gpt-4-1106-preview"},
    "gpt_4_0125_preview": {"source": "OAI", "full_name": "gpt-4-0125-preview"},
    "gpt_4o": {"source": "OAI", "full_name": "gpt-4o"},
    "llama_2_70b": {
        "source": "TOGETHER",
        "full_name": "meta-llama/Llama-2-70b-chat-hf",
    },
    "llama_3_8b": {
        "source": "TOGETHER",
        "full_name": "meta-llama/Llama-3-8b-chat-hf",
    },
    "llama_3_70b": {
        "source": "TOGETHER",
        "full_name": "meta-llama/Llama-3-70b-chat-hf",
    },
    "mistral_8x7b_instruct": {
        "source": "TOGETHER",
        "full_name": "mistralai/Mixtral-8x7B-Instruct-v0.1",
    },
    "mistral_8x22b_instruct": {
        "source": "TOGETHER",
        "full_name": "mistralai/Mixtral-8x22B-Instruct-v0.1",
    },
    "mistral_large": {
        "source": "MISTRAL",
        "full_name": "mistral-large-latest",
    },
    "qwen_1p5_110b": {
        "source": "TOGETHER",
        "full_name": "Qwen/Qwen1.5-110B-Chat",
    },
    "claude_2p1": {"source": "ANTHROPIC", "full_name": "claude-2.1"},
    "claude_3_opus": {"source": "ANTHROPIC", "full_name": "claude-3-opus-20240229"},
    "claude_3_sonnet": {"source": "ANTHROPIC", "full_name": "claude-3-sonnet-20240229"},
    "claude_3_haiku": {"source": "ANTHROPIC", "full_name": "claude-3-haiku-20240307"},
    #     "gemini_pro": {"source": "GOOGLE", "full_name": "gemini-pro"},
}

### Load data

In [3]:
import json


def read_jsonl(file_path):
    data = []
    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            if line.strip():
                json_object = json.loads(line)
                data.append(json_object)
    return data


file_path = "2024-05-03-llm.jsonl"
questions = read_jsonl(file_path)

single_non_acled_questions = [
    q for q in questions if q["combination_of"] == "N/A" and q["source"] != "acled"
]
single_acled_questions = [
    q for q in questions if q["combination_of"] == "N/A" and q["source"] == "acled"
]
combo_questions = [q for q in questions if q["combination_of"] != "N/A" and q["source"] == "acled"]

In [4]:
combo_questions_unrolled = []

for q in combo_questions:
    for i in range(4):
        new_q = q.copy()
        new_q["combo_index"] = i

        combo_questions_unrolled.append(new_q)

In [5]:
len(single_non_acled_questions), len(single_acled_questions), len(combo_questions_unrolled)

(339, 162, 644)

### Zero Shot Eval

In [6]:
def worker(index, model_name, save_dict, questions_to_eval, rate_limit=False):
    if save_dict[index] != "":
        return

    logger.info(f"{model_name} - {index}")

    if rate_limit:
        start_time = datetime.now()

    if questions_to_eval[index]["source"] != "acled":
        prompt = ZERO_SHOT_MARKET_PROMPT.format(
            question=questions_to_eval[index]["question"],
            background=questions_to_eval[index]["background"]
            + "\n"
            + questions_to_eval[index]["source_resolution_criteria"],
            resolution_criteria=questions_to_eval[index]["resolution_criteria"],
            close_date=questions_to_eval[index]["source_close_datetime"],
        )
        response = model_eval.get_response_from_model(
            prompt=prompt,
            max_tokens=100,
            model_name=models[model_name]["full_name"],
            temperature=0,
            wait_time=30,
        )

        save_dict[index] = model_eval.extract_probability(response)

    else:
        all_resolution_dates = []
        for horizon in questions_to_eval[index]["forecast_horizons"]:
            resolution_date = datetime.fromisoformat(
                questions_to_eval[index]["freeze_datetime"]
            ) + timedelta(days=7 + horizon)
            resolution_date = resolution_date.isoformat()
            all_resolution_dates.append(resolution_date)

        if questions_to_eval[index]["combination_of"] == "N/A":
            prompt = ZERO_SHOT_NON_MARKET_PROMPT.format(
                question=questions_to_eval[index]["question"],
                background=questions_to_eval[index]["background"]
                + "\n"
                + questions_to_eval[index]["source_resolution_criteria"],
                resolution_criteria=questions_to_eval[index]["resolution_criteria"],
                freeze_datetime=questions_to_eval[index]["freeze_datetime"],
                value_at_freeze_datetime=questions_to_eval[index]["value_at_freeze_datetime"],
                value_at_freeze_datetime_explanation=questions_to_eval[index][
                    "value_at_freeze_datetime_explanation"
                ],
                list_of_resolution_dates=all_resolution_dates,
            )
            response = model_eval.get_response_from_model(
                prompt=prompt,
                max_tokens=100,
                model_name=models[model_name]["full_name"],
                temperature=0,
                wait_time=30,
            )

            save_dict[index] = model_eval.reformat_answers(
                response=response, prompt=prompt, question=questions_to_eval[index]
            )

        else:
            prompt = ZERO_SHOT_JOINT_QUESTION_PROMPT.format(
                human_prompt=human_joint_prompts[questions_to_eval[index]["combo_index"]],
                question_1=questions_to_eval[index]["combination_of"][0]["question"],
                question_2=questions_to_eval[index]["combination_of"][1]["question"],
                background_1=questions_to_eval[index]["combination_of"][0]["background"]
                + "\n"
                + questions_to_eval[index]["combination_of"][0]["source_resolution_criteria"],
                background_2=questions_to_eval[index]["combination_of"][1]["background"]
                + "\n"
                + questions_to_eval[index]["combination_of"][1]["source_resolution_criteria"],
                resolution_criteria_1=questions_to_eval[index]["combination_of"][0][
                    "resolution_criteria"
                ],
                resolution_criteria_2=questions_to_eval[index]["combination_of"][1][
                    "resolution_criteria"
                ],
                freeze_datetime_1=questions_to_eval[index]["combination_of"][0]["freeze_datetime"],
                freeze_datetime_2=questions_to_eval[index]["combination_of"][1]["freeze_datetime"],
                value_at_freeze_datetime_1=questions_to_eval[index]["combination_of"][0][
                    "value_at_freeze_datetime"
                ],
                value_at_freeze_datetime_2=questions_to_eval[index]["combination_of"][1][
                    "value_at_freeze_datetime"
                ],
                value_at_freeze_datetime_explanation_1=questions_to_eval[index]["combination_of"][
                    0
                ]["value_at_freeze_datetime_explanation"],
                value_at_freeze_datetime_explanation_2=questions_to_eval[index]["combination_of"][
                    1
                ]["value_at_freeze_datetime_explanation"],
                list_of_resolution_dates=all_resolution_dates,
            )

            response = model_eval.get_response_from_model(
                prompt=prompt,
                max_tokens=500,
                model_name=models[model_name]["full_name"],
                temperature=0,
                wait_time=30,
            )

            save_dict[index] = model_eval.reformat_answers(
                response=response, prompt=prompt, question=questions_to_eval[index]
            )

    logger.info(save_dict[index])

    if rate_limit:
        end_time = datetime.now()
        elapsed_time = (end_time - start_time).total_seconds()
        if elapsed_time < 1:
            time.sleep(
                1 - elapsed_time
            )  # Ensure at least 1 second per request to stay within rate limits

    return None


def executor(max_workers, model_name, save_dict, questions_to_eval, use_gemini=False):
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        worker_with_args = partial(
            worker, model_name=model_name, save_dict=save_dict, questions_to_eval=questions_to_eval
        )
        return list(executor.map(worker_with_args, range(len(questions_to_eval))))

In [7]:
results = {}
model_result_loaded = {}
models_to_test = list(models.keys())[:]

for question in [single_non_acled_questions, single_acled_questions, combo_questions_unrolled]:
    questions_to_eval = question
    if question[0]["source"] != "acled":
        test_type = "zero_shot/non_acled"
    elif question[0]["source"] == "acled" and question[0]["combination_of"] == "N/A":
        test_type = "zero_shot/acled"
    else:
        test_type = "zero_shot/combo"

    for model in models_to_test:
        if model not in model_result_loaded.keys():
            model_result_loaded[model] = {}
        model_result_loaded[model] = False

    for model in models_to_test:
        file_path = f"{test_type}/{model}.jsonl"
        if model not in results.keys():
            results[model] = {}
        try:
            results[model] = read_jsonl(file_path)
            model_result_loaded[model] = True  # Set flag to True if loaded successfully
        except:
            results[model] = {i: "" for i in range(len(questions_to_eval))}

    for model in models_to_test:
        file_path = f"{test_type}/{model}.jsonl"
        if not model_result_loaded[model]:
            executor_count = 50
            use_gemini = False
            if models[model]["source"] == "GOOGLE":
                executor_count = 1
                use_gemini = True

            executor(executor_count, model, results[model], questions_to_eval, use_gemini)

            current_model_forecasts = []
            for index in range(len(questions_to_eval)):
                if questions_to_eval[index]["source"] == "acled":
                    for forecast, horizon in zip(
                        results[model][index], questions_to_eval[index]["forecast_horizons"]
                    ):
                        current_forecast = {
                            "id": questions_to_eval[index]["id"],
                            "source": questions_to_eval[index]["source"],
                            "forecast": forecast,
                            "horizon": horizon,
                            "reasoning": None,
                        }
                        if questions_to_eval[index]["combination_of"] != "N/A":
                            combo_index = questions_to_eval[index]["combo_index"]
                            if combo_index == 0:
                                current_forecast["direction"] = [1, 1]
                            elif combo_index == 1:
                                current_forecast["direction"] = [1, -1]
                            elif combo_index == 2:
                                current_forecast["direction"] = [-1, 1]
                            else:
                                current_forecast["direction"] = [-1, -1]

                        current_model_forecasts.append(current_forecast)
                else:
                    current_forecast = {
                        "id": questions_to_eval[index]["id"],
                        "source": questions_to_eval[index]["source"],
                        "forecast": results[model][index],
                        "reasoning": None,
                    }
                    current_model_forecasts.append(current_forecast)

            os.makedirs(os.path.dirname(file_path), exist_ok=True)
            with open(file_path, "w") as file:
                for entry in current_model_forecasts:
                    json_line = json.dumps(entry)
                    file.write(json_line + "\n")

INFO:root:claude_2p1 - 0
INFO:root:claude_2p1 - 1
INFO:root:claude_2p1 - 2
INFO:root:claude_2p1 - 3
INFO:root:claude_2p1 - 4
INFO:root:claude_2p1 - 5
INFO:root:claude_2p1 - 6
INFO:root:claude_2p1 - 7
INFO:root:claude_2p1 - 8
INFO:root:claude_2p1 - 9
INFO:root:claude_2p1 - 10
INFO:root:claude_2p1 - 11
INFO:root:claude_2p1 - 12
INFO:root:claude_2p1 - 13
INFO:root:claude_2p1 - 14
INFO:root:claude_2p1 - 15
INFO:root:claude_2p1 - 16
INFO:root:claude_2p1 - 17
INFO:root:claude_2p1 - 18
INFO:root:claude_2p1 - 19
INFO:root:claude_2p1 - 20
INFO:root:claude_2p1 - 21
INFO:root:claude_2p1 - 22
INFO:root:claude_2p1 - 23
INFO:root:claude_2p1 - 24
INFO:root:claude_2p1 - 25
INFO:root:claude_2p1 - 26
INFO:root:claude_2p1 - 27
INFO:root:claude_2p1 - 28
INFO:root:claude_2p1 - 29
INFO:root:claude_2p1 - 30
INFO:root:claude_2p1 - 31
INFO:root:claude_2p1 - 32
INFO:root:claude_2p1 - 33
INFO:root:claude_2p1 - 34
INFO:root:claude_2p1 - 35
INFO:root:claude_2p1 - 36
INFO:root:claude_2p1 - 37
INFO:root:claude_2p1 -

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
INFO:root:[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
INFO:root:claude_2p1 - 52
INFO:root:claude_2p1 - 53
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:claude_2p1 - 54
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
INFO:root:claude_2p1 - 55
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[0.5, 0.5, 0.5, 0.5, 

INFO:root:claude_2p1 - 89
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
INFO:root:claude_2p1 - 90
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:claude_2p1 - 91
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:claude_2p1 - 92
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[0.21415125278761601, 0.21415125278761601, 0.21415125278761601, 0.21415125278761601, 0.2

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:claude_2p1 - 103
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:claude_2p1 - 104
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ap

INFO:root:[0.17, 0.17, 0.17, 0.17, 0.17, 0.17, 0.17, 0.17]
INFO:root:claude_2p1 - 135
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
INFO:root:claude_2p1 - 136
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[0.22199031279491002, 0.22199031279491002, 0.22199031279491002, 0.22199031279491002, 0.22199031279491002, 0.22199031279491002, 0.22199031279491002, 0.22199031279491002]
INFO:root:claude_2p1 - 137
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]
INFO:root:claude_2p1 - 138
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK

...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:claude_2p1 - 586
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:claude_2p1 - 587
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:claude_2p1 - 588
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:root:[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.

INFO:root:claude_2p1 - 608
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:claude_2p1 - 609
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:claude_2p1 - 610
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25]
INFO:root:claude_2p1 - 611
INFO:httpx:HTTP Re

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request:

INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST ht

INFO:root:[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/c

In [8]:
prompt_type = "zero_shot"

for model in models_to_test:
    current_model_forecasts = []
    for test_type in [f"{prompt_type}/non_acled", f"{prompt_type}/acled", f"{prompt_type}/combo"]:
        file_path = f"{test_type}/{model}.jsonl"
        questions = read_jsonl(file_path)
        current_model_forecasts.extend(questions)

    final_file_name = f"{prompt_type}/final/{model}"
    os.makedirs(os.path.dirname(final_file_name), exist_ok=True)
    with open(final_file_name, "w") as file:
        for entry in current_model_forecasts:
            json_line = json.dumps(entry)
            file.write(json_line + "\n")

for model in models_to_test:
    file_path = f"{prompt_type}/final/{model}"
    questions = read_jsonl(file_path)
    if "gpt" in model:
        org = "OPENAI"
    elif "llama" in model:
        org = "META"
    elif "mistral" in model:
        org = "MISTRAL"
    elif "claude" in model:
        org = "ANTRHOPIC"
    elif "qwen" in model:
        org = "Qwen"

    directory = f"{prompt_type}/final_submit"
    os.makedirs(directory, exist_ok=True)

    new_file_name = f"{directory}/2024-05-03.{org}.{model}_{prompt_type}.json"

    forecast_file = {
        "organization": org,
        "model": f"{model.replace('_', ' ')} {prompt_type.replace('_', ' ')}",
        "question_set": "2024-05-03-llm.jsonl",
        "forecast_date": "2024-05-03",
        "forecasts": questions,
    }

    with open(new_file_name, "w") as f:
        json.dump(forecast_file, f, indent=4)

### Scratchpad

In [9]:
def worker(index, model_name, save_dict, questions_to_eval):
    if save_dict[index] != "":
        return

    logger.info(f"Starting {model_name} - {index}")

    if questions_to_eval[index]["source"] != "acled":
        prompt = SCRATCH_PAD_MARKET_PROMPT.format(
            question=questions_to_eval[index]["question"],
            background=questions_to_eval[index]["background"]
            + "\n"
            + questions_to_eval[index]["source_resolution_criteria"],
            resolution_criteria=questions_to_eval[index]["resolution_criteria"],
            close_date=questions_to_eval[index]["source_close_datetime"],
        )
        response = model_eval.get_response_from_model(
            prompt=prompt,
            max_tokens=1300,
            model_name=models[model_name]["full_name"],
            temperature=0,
            wait_time=30,
        )

        save_dict[index] = (model_eval.reformat_answers(response=response, single=True), response)

    else:
        all_resolution_dates = []
        for horizon in questions_to_eval[index]["forecast_horizons"]:
            resolution_date = datetime.fromisoformat(
                questions_to_eval[index]["freeze_datetime"]
            ) + timedelta(days=7 + horizon)
            resolution_date = resolution_date.isoformat()
            all_resolution_dates.append(resolution_date)

        if questions_to_eval[index]["combination_of"] == "N/A":
            prompt = SCRATCH_PAD_NON_MARKET_PROMPT.format(
                question=questions_to_eval[index]["question"],
                background=questions_to_eval[index]["background"]
                + "\n"
                + questions_to_eval[index]["source_resolution_criteria"],
                resolution_criteria=questions_to_eval[index]["resolution_criteria"],
                freeze_datetime=questions_to_eval[index]["freeze_datetime"],
                value_at_freeze_datetime=questions_to_eval[index]["value_at_freeze_datetime"],
                value_at_freeze_datetime_explanation=questions_to_eval[index][
                    "value_at_freeze_datetime_explanation"
                ],
                list_of_resolution_dates=all_resolution_dates,
            )
            response = model_eval.get_response_from_model(
                prompt=prompt,
                max_tokens=2000,
                model_name=models[model_name]["full_name"],
                temperature=0,
                wait_time=30,
            )
            save_dict[index] = (
                model_eval.reformat_answers(
                    response=response, prompt=prompt, question=questions_to_eval[index]
                ),
                response,
            )
        else:
            prompt = SCRATCH_PAD_JOINT_QUESTION_PROMPT.format(
                human_prompt=human_joint_prompts[questions_to_eval[index]["combo_index"]],
                question_1=questions_to_eval[index]["combination_of"][0]["question"],
                question_2=questions_to_eval[index]["combination_of"][1]["question"],
                background_1=questions_to_eval[index]["combination_of"][0]["background"]
                + "\n"
                + questions_to_eval[index]["combination_of"][0]["source_resolution_criteria"],
                background_2=questions_to_eval[index]["combination_of"][1]["background"]
                + "\n"
                + questions_to_eval[index]["combination_of"][1]["source_resolution_criteria"],
                resolution_criteria_1=questions_to_eval[index]["combination_of"][0][
                    "resolution_criteria"
                ],
                resolution_criteria_2=questions_to_eval[index]["combination_of"][1][
                    "resolution_criteria"
                ],
                freeze_datetime_1=questions_to_eval[index]["combination_of"][0]["freeze_datetime"],
                freeze_datetime_2=questions_to_eval[index]["combination_of"][1]["freeze_datetime"],
                value_at_freeze_datetime_1=questions_to_eval[index]["combination_of"][0][
                    "value_at_freeze_datetime"
                ],
                value_at_freeze_datetime_2=questions_to_eval[index]["combination_of"][1][
                    "value_at_freeze_datetime"
                ],
                value_at_freeze_datetime_explanation_1=questions_to_eval[index]["combination_of"][
                    0
                ]["value_at_freeze_datetime_explanation"],
                value_at_freeze_datetime_explanation_2=questions_to_eval[index]["combination_of"][
                    1
                ]["value_at_freeze_datetime_explanation"],
                list_of_resolution_dates=all_resolution_dates,
            )

            response = model_eval.get_response_from_model(
                prompt=prompt,
                max_tokens=2000,
                model_name=models[model_name]["full_name"],
                temperature=0,
                wait_time=30,
            )

            save_dict[index] = (
                model_eval.reformat_answers(
                    response=response, prompt=prompt, question=questions_to_eval[index]
                ),
                response,
            )

    logger.info(f"Answer {save_dict[index][0]}")

    return None


def executor(max_workers, model_name, save_dict, questions_to_eval):
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        worker_with_args = partial(
            worker, model_name=model_name, save_dict=save_dict, questions_to_eval=questions_to_eval
        )
        return list(executor.map(worker_with_args, range(len(questions_to_eval))))

In [10]:
results = {}
model_result_loaded = {}
models_to_test = list(models.keys())[:]


for question in [single_acled_questions, single_non_acled_questions, combo_questions_unrolled]:
    questions_to_eval = question
    if question[0]["source"] != "acled":
        test_type = "scratchpad/non_acled"
    elif question[0]["source"] == "acled" and question[0]["combination_of"] == "N/A":
        test_type = "scratchpad/acled"
    else:
        test_type = "scratchpad/combo"

    for model in models_to_test:
        if model not in model_result_loaded.keys():
            model_result_loaded[model] = {}
        model_result_loaded[model] = False

    for model in models_to_test:
        file_path = f"{test_type}/{model}.jsonl"
        if model not in results.keys():
            results[model] = {}
        try:
            results[model] = read_jsonl(file_path)
            model_result_loaded[model] = True  # Set flag to True if loaded successfully
        except:
            results[model] = {i: "" for i in range(len(questions_to_eval))}

    for model in models_to_test:
        file_path = f"{test_type}/{model}.jsonl"
        if not model_result_loaded[model]:
            executor_count = 50
            if models[model]["source"] == "ANTHROPIC":
                executor_count = 30

            executor(executor_count, model, results[model], questions_to_eval)

            current_model_forecasts = []
            for index in range(len(questions_to_eval)):
                if questions_to_eval[index]["source"] == "acled":
                    for forecast, horizon in zip(
                        results[model][index][0], questions_to_eval[index]["forecast_horizons"]
                    ):
                        current_forecast = {
                            "id": questions_to_eval[index]["id"],
                            "source": questions_to_eval[index]["source"],
                            "forecast": forecast,
                            "horizon": horizon,
                            "reasoning": results[model][index][1],
                        }

                        if questions_to_eval[index]["combination_of"] != "N/A":
                            combo_index = questions_to_eval[index]["combo_index"]
                            if combo_index == 0:
                                current_forecast["direction"] = [1, 1]
                            elif combo_index == 1:
                                current_forecast["direction"] = [1, -1]
                            elif combo_index == 2:
                                current_forecast["direction"] = [-1, 1]
                            else:
                                current_forecast["direction"] = [-1, -1]

                        current_model_forecasts.append(current_forecast)

                else:
                    current_forecast = {
                        "id": questions_to_eval[index]["id"],
                        "source": questions_to_eval[index]["source"],
                        "forecast": results[model][index][0],
                        "reasoning": results[model][index][1],
                    }
                    current_model_forecasts.append(current_forecast)

            os.makedirs(os.path.dirname(file_path), exist_ok=True)
            with open(file_path, "w") as file:
                for entry in current_model_forecasts:
                    json_line = json.dumps(entry)
                    file.write(json_line + "\n")

INFO:root:Starting claude_2p1 - 0
INFO:root:Starting claude_2p1 - 1
INFO:root:Starting claude_2p1 - 2
INFO:root:Starting claude_2p1 - 3
INFO:root:Starting claude_2p1 - 4
INFO:root:Starting claude_2p1 - 5
INFO:root:Starting claude_2p1 - 6
INFO:root:Starting claude_2p1 - 7
INFO:root:Starting claude_2p1 - 8
INFO:root:Starting claude_2p1 - 9
INFO:root:Starting claude_2p1 - 10
INFO:root:Starting claude_2p1 - 11
INFO:root:Starting claude_2p1 - 12
INFO:root:Starting claude_2p1 - 13
INFO:root:Starting claude_2p1 - 14
INFO:root:Starting claude_2p1 - 15
INFO:root:Starting claude_2p1 - 16
INFO:root:Starting claude_2p1 - 17
INFO:root:Starting claude_2p1 - 18
INFO:root:Starting claude_2p1 - 19
INFO:root:Starting claude_2p1 - 20
INFO:root:Starting claude_2p1 - 21
INFO:root:Starting claude_2p1 - 22
INFO:root:Starting claude_2p1 - 23
INFO:root:Starting claude_2p1 - 24
INFO:root:Starting claude_2p1 - 25
INFO:root:Starting claude_2p1 - 26
INFO:root:Starting claude_2p1 - 27
INFO:root:Starting claude_2p1 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer [0.6, 0.4, 0.7, 0.3, 0.5, 0.6, 0.4, 0.7]
INFO:root:Starting claude_2p1 - 47
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]
INFO:root:Starting claude_2p1 - 48
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:Starting claude_2p1 - 49
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INF

INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:Starting claude_2p1 - 70
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:Starting claude_2p1 - 71
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]
INFO:root:Starting claude_2p1 - 72
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
INFO:root:Starting claude_2p1 - 73
I

INFO:root:Answer [0.01, 0.01, 0.01, 0.01, 0.05, 0.1, 0.15, 0.2]
INFO:root:Starting claude_2p1 - 93
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
INFO:root:Starting claude_2p1 - 94
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer [0.5, 0.4, 0.6, 0.3, 0.7, 0.2, 0.4, 0.5]
INFO:root:Starting claude_2p1 - 95
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:Starting claude_2p1 - 96
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer [0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8]
INFO:root:Starting 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:Starting claude_2p1 - 116
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer [0.5, 0.4, 0.6, 0.3, 0.7, 0.8, 0.9, 0.2]
INFO:root:Starting claude_2p1 - 117
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer [0.5, 0.4, 0.6, 0.3, 0.7, 0.4, 0.5, 0.6]
INFO:root:Starting claude_2p1 - 118
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
INFO:root:Starting clau

...

INFO:root:Answer [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:Starting claude_2p1 - 557
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]
INFO:root:Starting claude_2p1 - 558
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:Starting claude_2p1 - 559
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:Starting claude

INFO:root:Answer [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:Starting claude_2p1 - 581
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:Starting claude_2p1 - 582
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Re

INFO:root:Answer [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:Starting claude_2p1 - 599
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:Starting claude_2p1 - 600
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer [0.6, 0.55, 0.7, 0.65, 0.5, 0.4, 0.45, 0.55]
INFO:root:Starting claude_2p1 - 601
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer [0.45, 0.5, 0.48, 0.52, 0.47, 0.49, 0.51, 0.46]
INFO:root:St

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:Starting claude_2p1 - 625
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PO

In [11]:
prompt_type = "scratchpad"

for model in models_to_test:
    current_model_forecasts = []
    for test_type in [f"{prompt_type}/non_acled", f"{prompt_type}/acled", f"{prompt_type}/combo"]:
        file_path = f"{test_type}/{model}.jsonl"
        questions = read_jsonl(file_path)
        current_model_forecasts.extend(questions)

    final_file_name = f"{prompt_type}/final/{model}"
    os.makedirs(os.path.dirname(final_file_name), exist_ok=True)
    with open(final_file_name, "w") as file:
        for entry in current_model_forecasts:
            json_line = json.dumps(entry)
            file.write(json_line + "\n")

for model in models_to_test:
    file_path = f"{prompt_type}/final/{model}"
    questions = read_jsonl(file_path)
    if "gpt" in model:
        org = "OPENAI"
    elif "llama" in model:
        org = "META"
    elif "mistral" in model:
        org = "MISTRAL"
    elif "claude" in model:
        org = "ANTRHOPIC"
    elif "qwen" in model:
        org = "Qwen"

    directory = f"{prompt_type}/final_submit"
    os.makedirs(directory, exist_ok=True)

    new_file_name = f"{directory}/2024-05-03.{org}.{model}_{prompt_type}.json"

    forecast_file = {
        "organization": org,
        "model": f"{model.replace('_', ' ')} {prompt_type.replace('_', ' ')}",
        "question_set": "2024-05-03-llm.jsonl",
        "forecast_date": "2024-05-03",
        "forecasts": questions,
    }

    with open(new_file_name, "w") as f:
        json.dump(forecast_file, f, indent=4)

### Scratchpad + Retrieval

In [12]:
models = {
    "gpt_3p5_turbo_0125": {"source": "OAI", "full_name": "gpt-3.5-turbo-0125"},
    "gpt_4_turbo_0409": {"source": "OAI", "full_name": "gpt-4-turbo-2024-04-09"},
    "gpt_4_1106_preview": {"source": "OAI", "full_name": "gpt-4-1106-preview"},
    "gpt_4_0125_preview": {"source": "OAI", "full_name": "gpt-4-0125-preview"},
    "gpt_4o": {"source": "OAI", "full_name": "gpt-4o"},
    "mistral_8x7b_instruct": {
        "source": "TOGETHER",
        "full_name": "mistralai/Mixtral-8x7B-Instruct-v0.1",
    },
    "mistral_8x22b_instruct": {
        "source": "TOGETHER",
        "full_name": "mistralai/Mixtral-8x22B-Instruct-v0.1",
    },
    "mistral_large": {
        "source": "MISTRAL",
        "full_name": "mistral-large-latest",
    },
    "qwen_1p5_110b": {
        "source": "TOGETHER",
        "full_name": "Qwen/Qwen1.5-110B-Chat",
    },
    "claude_2p1": {"source": "ANTHROPIC", "full_name": "claude-2.1"},
    "claude_3_opus": {"source": "ANTHROPIC", "full_name": "claude-3-opus-20240229"},
    "claude_3_sonnet": {"source": "ANTHROPIC", "full_name": "claude-3-sonnet-20240229"},
    "claude_3_haiku": {"source": "ANTHROPIC", "full_name": "claude-3-haiku-20240307"},
}

In [13]:
# Mapping retrieved summaries back into questions
for question_source in [single_non_acled_questions, single_acled_questions]:
    for q in question_source:
        reformatted_id = q["id"].replace("/", "_")
        filename = f"info_retrieval/{reformatted_id}.pickle"
        with open(filename, "rb") as file:
            retrieved_info = pickle.load(file)
        q["info_retrieval"] = retrieved_info

for q in combo_questions_unrolled:
    for sub_q in q["combination_of"]:
        reformatted_id = sub_q["id"].replace("/", "_")
        filename = f"info_retrieval/{reformatted_id}.pickle"
        with open(filename, "rb") as file:
            retrieved_info = pickle.load(file)
        sub_q["info_retrieval"] = retrieved_info

In [14]:
def get_all_retrieved_info(all_retrieved_info):
    retrieved_info = ""
    for summary in all_retrieved_info:
        retrieved_info += f"Article title: {summary['title']}" + "\n"
        retrieved_info += f"Summary: {summary['summary']}" + "\n\n"
    return retrieved_info


def worker(index, model_name, save_dict, questions_to_eval):
    if save_dict[index] != "":
        return

    logger.info(f"Starting {model_name} - {index}")

    if questions_to_eval[index]["source"] != "acled":
        prompt = SCRATCH_PAD_WITH_SUMMARIES_MARKET_PROMPT.format(
            question=questions_to_eval[index]["question"],
            background=questions_to_eval[index]["background"]
            + "\n"
            + questions_to_eval[index]["source_resolution_criteria"],
            resolution_criteria=questions_to_eval[index]["resolution_criteria"],
            close_date=questions_to_eval[index]["source_close_datetime"],
            retrieved_info=get_all_retrieved_info(questions_to_eval[index]["info_retrieval"]),
        )
        response = model_eval.get_response_from_model(
            prompt=prompt,
            max_tokens=2000,
            model_name=models[model_name]["full_name"],
            temperature=0,
            wait_time=30,
        )

        save_dict[index] = (model_eval.reformat_answers(response=response, single=True), response)

    else:
        all_resolution_dates = []
        for horizon in questions_to_eval[index]["forecast_horizons"]:
            resolution_date = datetime.fromisoformat(
                questions_to_eval[index]["freeze_datetime"]
            ) + timedelta(days=7 + horizon)
            resolution_date = resolution_date.isoformat()
            all_resolution_dates.append(resolution_date)

        if questions_to_eval[index]["combination_of"] == "N/A":
            prompt = SCRATCH_PAD_WITH_SUMMARIES_NON_MARKET_PROMPT.format(
                question=questions_to_eval[index]["question"],
                background=questions_to_eval[index]["background"]
                + "\n"
                + questions_to_eval[index]["source_resolution_criteria"],
                resolution_criteria=questions_to_eval[index]["resolution_criteria"],
                freeze_datetime=questions_to_eval[index]["freeze_datetime"],
                value_at_freeze_datetime=questions_to_eval[index]["value_at_freeze_datetime"],
                value_at_freeze_datetime_explanation=questions_to_eval[index][
                    "value_at_freeze_datetime_explanation"
                ],
                retrieved_info=get_all_retrieved_info(questions_to_eval[index]["info_retrieval"]),
                list_of_resolution_dates=all_resolution_dates,
            )
            response = model_eval.get_response_from_model(
                prompt=prompt,
                max_tokens=2000,
                model_name=models[model_name]["full_name"],
                temperature=0,
                wait_time=30,
            )
            save_dict[index] = (
                model_eval.reformat_answers(
                    response=response, prompt=prompt, question=questions_to_eval[index]
                ),
                response,
            )
        else:
            prompt = SCRATCH_PAD_WITH_SUMMARIES_JOINT_QUESTION_PROMPT.format(
                human_prompt=human_joint_prompts[questions_to_eval[index]["combo_index"]],
                question_1=questions_to_eval[index]["combination_of"][0]["question"],
                question_2=questions_to_eval[index]["combination_of"][1]["question"],
                background_1=questions_to_eval[index]["combination_of"][0]["background"]
                + "\n"
                + questions_to_eval[index]["combination_of"][0]["source_resolution_criteria"],
                background_2=questions_to_eval[index]["combination_of"][1]["background"]
                + "\n"
                + questions_to_eval[index]["combination_of"][1]["source_resolution_criteria"],
                resolution_criteria_1=questions_to_eval[index]["combination_of"][0][
                    "resolution_criteria"
                ],
                resolution_criteria_2=questions_to_eval[index]["combination_of"][1][
                    "resolution_criteria"
                ],
                freeze_datetime_1=questions_to_eval[index]["combination_of"][0]["freeze_datetime"],
                freeze_datetime_2=questions_to_eval[index]["combination_of"][1]["freeze_datetime"],
                value_at_freeze_datetime_1=questions_to_eval[index]["combination_of"][0][
                    "value_at_freeze_datetime"
                ],
                value_at_freeze_datetime_2=questions_to_eval[index]["combination_of"][1][
                    "value_at_freeze_datetime"
                ],
                value_at_freeze_datetime_explanation_1=questions_to_eval[index]["combination_of"][
                    0
                ]["value_at_freeze_datetime_explanation"],
                value_at_freeze_datetime_explanation_2=questions_to_eval[index]["combination_of"][
                    1
                ]["value_at_freeze_datetime_explanation"],
                retrieved_info_1=get_all_retrieved_info(
                    questions_to_eval[index]["combination_of"][0]["info_retrieval"]
                ),
                retrieved_info_2=get_all_retrieved_info(
                    questions_to_eval[index]["combination_of"][1]["info_retrieval"]
                ),
                list_of_resolution_dates=all_resolution_dates,
            )

            response = model_eval.get_response_from_model(
                prompt=prompt,
                max_tokens=2000,
                model_name=models[model_name]["full_name"],
                temperature=0,
                wait_time=30,
            )

            save_dict[index] = (
                model_eval.reformat_answers(
                    response=response, prompt=prompt, question=questions_to_eval[index]
                ),
                response,
            )

    logger.info(f"Answer: {save_dict[index][0]}")

    return None


def executor(max_workers, model_name, save_dict, questions_to_eval):
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        worker_with_args = partial(
            worker, model_name=model_name, save_dict=save_dict, questions_to_eval=questions_to_eval
        )
        return list(executor.map(worker_with_args, range(len(questions_to_eval))))

In [15]:
results = {}
model_result_loaded = {}
models_to_test = list(models.keys())[:]


for question in [single_acled_questions, combo_questions_unrolled, single_non_acled_questions]:
    questions_to_eval = question
    if question[0]["source"] != "acled":
        test_type = "scratchpad_with_info_retrieval/non_acled"
    elif question[0]["source"] == "acled" and question[0]["combination_of"] == "N/A":
        test_type = "scratchpad_with_info_retrieval/acled"
    else:
        test_type = "scratchpad_with_info_retrieval/combo"

    for model in models_to_test:
        if model not in model_result_loaded.keys():
            model_result_loaded[model] = {}
        model_result_loaded[model] = False

    for model in models_to_test:
        file_path = f"{test_type}/{model}.jsonl"
        if model not in results.keys():
            results[model] = {}
        try:
            results[model] = read_jsonl(file_path)
            model_result_loaded[model] = True  # Set flag to True if loaded successfully
        except:
            results[model] = {i: "" for i in range(len(questions_to_eval))}

    for model in models_to_test:
        file_path = f"{test_type}/{model}.jsonl"
        if not model_result_loaded[model]:
            executor_count = 50
            if models[model]["source"] == "ANTHROPIC":
                executor_count = 30

            executor(executor_count, model, results[model], questions_to_eval)

            current_model_forecasts = []
            for index in range(len(questions_to_eval)):
                if questions_to_eval[index]["source"] == "acled":
                    for forecast, horizon in zip(
                        results[model][index][0], questions_to_eval[index]["forecast_horizons"]
                    ):
                        current_forecast = {
                            "id": questions_to_eval[index]["id"],
                            "source": questions_to_eval[index]["source"],
                            "forecast": forecast,
                            "horizon": horizon,
                            "reasoning": results[model][index][1],
                        }

                        if questions_to_eval[index]["combination_of"] != "N/A":
                            combo_index = questions_to_eval[index]["combo_index"]
                            if combo_index == 0:
                                current_forecast["direction"] = [1, 1]
                            elif combo_index == 1:
                                current_forecast["direction"] = [1, -1]
                            elif combo_index == 2:
                                current_forecast["direction"] = [-1, 1]
                            else:
                                current_forecast["direction"] = [-1, -1]

                        current_model_forecasts.append(current_forecast)

                else:
                    current_forecast = {
                        "id": questions_to_eval[index]["id"],
                        "source": questions_to_eval[index]["source"],
                        "forecast": results[model][index][0],
                        "reasoning": results[model][index][1],
                    }
                    current_model_forecasts.append(current_forecast)

            os.makedirs(os.path.dirname(file_path), exist_ok=True)
            with open(file_path, "w") as file:
                for entry in current_model_forecasts:
                    json_line = json.dumps(entry)
                    file.write(json_line + "\n")

INFO:root:Starting claude_2p1 - 0
INFO:root:Starting claude_2p1 - 1
INFO:root:Starting claude_2p1 - 2
INFO:root:Starting claude_2p1 - 3
INFO:root:Starting claude_2p1 - 4
INFO:root:Starting claude_2p1 - 17
INFO:root:Starting claude_2p1 - 6
INFO:root:Starting claude_2p1 - 7
INFO:root:Starting claude_2p1 - 8
INFO:root:Starting claude_2p1 - 9
INFO:root:Starting claude_2p1 - 10
INFO:root:Starting claude_2p1 - 11
INFO:root:Starting claude_2p1 - 12
INFO:root:Starting claude_2p1 - 13
INFO:root:Starting claude_2p1 - 14
INFO:root:Starting claude_2p1 - 15
INFO:root:Starting claude_2p1 - 16
INFO:root:Starting claude_2p1 - 5
INFO:root:Starting claude_2p1 - 18
INFO:root:Starting claude_2p1 - 19
INFO:root:Starting claude_2p1 - 20
INFO:root:Starting claude_2p1 - 21
INFO:root:Starting claude_2p1 - 22
INFO:root:Starting claude_2p1 - 23
INFO:root:Starting claude_2p1 - 24
INFO:root:Starting claude_2p1 - 25
INFO:root:Starting claude_2p1 - 26
INFO:root:Starting claude_2p1 - 27
INFO:root:Starting claude_2p1 

INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer: [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:Starting claude_2p1 - 48
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer: [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:Starting claude_2p1 - 49
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HT

INFO:root:Answer: [0.55, 0.55, 0.55, 0.55, 0.5, 0.45, 0.4, 0.3]
INFO:root:Starting claude_2p1 - 70
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer: [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:Starting claude_2p1 - 71
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer: [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:Starting claude_2p1 - 72
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer: [0.1, 0.15, 0.1, 0.05, 0.02, 0.02, 0.02, 0.01]
INFO:root:Starting claude_2p1 - 73
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer: [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:Starting claude_2p1 - 93
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer: [0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]
INFO:root:Starting claude_2p1 - 94
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INF

INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer: [0.6, 0.5, 0.4, 0.55, 0.45, 0.6, 0.5, 0.55]
INFO:root:Starting claude_2p1 - 117
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer: [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]
INFO:root:Starting claude_2p1 - 118
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer: [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
INFO:root:Starting cla

...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer: 0.55
INFO:root:Starting claude_2p1 - 244
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer: 0.55
INFO:root:Starting claude_2p1 - 245
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer: 0.3
INFO:root:Starting claude_2p1 - 246
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer: No

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer: 0.55
INFO:root:Starting claude_2p1 - 270
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:root:Answer: 0.6
INFO:root:Starting claude_2p1 - 271
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer: 0.1
INFO:root:Starting claude_2p1 - 272
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP 

INFO:root:Answer: 0.4
INFO:root:Starting claude_2p1 - 291
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer: 0.25
INFO:root:Starting claude_2p1 - 292
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer: 0.3
INFO:root:Starting claude_2p1 - 293
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: P

INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer: None
INFO:root:Starting claude_2p1 - 317
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer: 0.6
INFO:root:Starting claude_2p1 - 318
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer: 0.65
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer: 0.55
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Answer: 0.65
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https:

In [16]:
prompt_type = "scratchpad_with_info_retrieval"

for model in models_to_test:
    current_model_forecasts = []
    for test_type in [f"{prompt_type}/non_acled", f"{prompt_type}/acled", f"{prompt_type}/combo"]:
        file_path = f"{test_type}/{model}.jsonl"
        questions = read_jsonl(file_path)
        current_model_forecasts.extend(questions)

    final_file_name = f"{prompt_type}/final/{model}"
    os.makedirs(os.path.dirname(final_file_name), exist_ok=True)
    with open(final_file_name, "w") as file:
        for entry in current_model_forecasts:
            json_line = json.dumps(entry)
            file.write(json_line + "\n")

for model in models_to_test:
    file_path = f"{prompt_type}/final/{model}"
    questions = read_jsonl(file_path)
    if "gpt" in model:
        org = "OPENAI"
    elif "llama" in model:
        org = "META"
    elif "mistral" in model:
        org = "MISTRAL"
    elif "claude" in model:
        org = "ANTRHOPIC"
    elif "qwen" in model:
        org = "Qwen"

    directory = f"{prompt_type}/final_submit"
    os.makedirs(directory, exist_ok=True)

    new_file_name = f"{directory}/2024-05-03.{org}.{model}_{prompt_type}.json"

    forecast_file = {
        "organization": org,
        "model": f"{model.replace('_', ' ')} {prompt_type.replace('_', ' ')}",
        "question_set": "2024-05-03-llm.jsonl",
        "forecast_date": "2024-05-03",
        "forecasts": questions,
    }

    with open(new_file_name, "w") as f:
        json.dump(forecast_file, f, indent=4)